# Dataloader

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error,median_absolute_error,mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
import matplotlib.pyplot as plt
from itertools import permutations
from itertools import combinations 
%matplotlib inline

In [2]:
# Load states data
state_data = pd.DataFrame()

# Insert relevant path here
state_csv = '/content/drive/MyDrive/MIT_path_check/predict_daily_cases/state.csv'
state_data = pd.read_csv(state_csv, low_memory=False)

print(state_data.shape)
state_data_dropped = state_data.dropna()# na values dropped # may be fill na 
print(state_data_dropped.shape)
state_data_dropped = state_data_dropped[(state_data_dropped['gender']=='overall') & (state_data_dropped['age_bucket']=='overall')]
print(state_data_dropped.shape)

(83404, 104)
(76790, 104)
(7441, 104)


# Preprocessing

In [3]:
y = state_data_dropped['pct_tested_and_positive']
output_like_variables = ['pct_tested_and_positive','pct_tested_and_negative', 'pct_tested_no_result', 'pct_could_not_get_tested', 'pct_did_not_try_to_get_tested',\
                       'pct_tested_and_positive_weighted','pct_contact_covid_positive', 'pct_contact_covid_positive_weighted' ,\
                       'pct_tested_and_negative_weighted',	'pct_tested_no_result_weighted',	'pct_could_not_get_tested_weighted',	'pct_did_not_try_to_get_tested_weighted'] 
                        
X = state_data_dropped.drop(output_like_variables,axis=1)

unweighted = ['n','weight_sums','pct_cli','pct_ili', 'pct_cli_anosmia_ageusia',\
        'pct_hh_cli', 'pct_cmnty_cli', 'pct_hh_fever', 'pct_hh_sore_throat',\
         'pct_hh_cough', 'pct_hh_shortness_of_breath','pct_hh_difficulty_breathing', 'mean_hh_cli_ct', 'mean_cmnty_cli_ct',\
         'pct_self_fever', 'pct_self_cough', 'pct_self_shortness_of_breath',\
         'pct_self_difficulty_breathing', 'pct_self_tiredness_or_exhaustion',\
         'pct_self_nasal_congestion', 'pct_self_runny_nose',\
         'pct_self_muscle_joint_aches', 'pct_self_sore_throat',
       'pct_self_persistent_pain_pressure_in_chest',\
       'pct_self_nausea_vomiting', 'pct_self_diarrhea',\
       'pct_self_anosmia_ageusia', 'pct_self_other', 'pct_self_none_of_above',\
       'pct_self_multiple_symptoms', 'pct_worked_outside_home',\
       'pct_avoid_contact_all_or_most_time',\
       'mean_outside_hh_contact_at_work_ct',\
       'mean_outside_hh_contact_shopping_ct',\
       'mean_outside_hh_contact_in_social_gatherings_ct', 'pct_diabetes',\
       'pct_cancer', 'pct_heart_disease', 'pct_high_blood_pressure', \
       'pct_asthma', 'pct_chronic_lung_disease', 'pct_kidney_disease',\
       'pct_autoimmune_disorder', 'pct_no_above_medical_conditions',\
       'pct_multiple_medical_conditions', ]

X = X.drop(unweighted,axis=1)
n_features = X.shape[1]
print(n_features, X.shape)

derived = ['pct_ili_weighted','pct_cli_weighted','pct_cli_anosmia_ageusia_weighted',]
X = X.drop(derived,axis=1)
n_features = X.shape[1]

mean_D = ['mean_outside_hh_contact_in_social_gatherings_ct_weighted','mean_cmnty_cli_ct_weighted',\
        'mean_outside_hh_contact_shopping_ct_weighted','mean_outside_hh_contact_at_work_ct_weighted']
X = X.drop(mean_D,axis=1)
n_features = X.shape[1]
print(n_features, X.shape)



47 (7441, 47)
40 (7441, 40)


In [4]:
who = pd.read_csv('/content/drive/MyDrive/MIT_path_check/predict_daily_cases/refined-us-states.csv')
print(who.info())
who = who.rename(columns={"state": "state_code"})

who['date'] = list(map(lambda x : '-'.join(list(reversed(x.split('-')))) ,list(who['date'])) )
print(who.head())


merged = pd.merge(X, who,  how='inner', on=['date','state_code'] )
merged = merged.sort_values(['state_code', 'date'], ascending=[True, True])
print(len(merged))
print('num states: ', merged['state_code'].nunique())
merged.head()

merged = merged.drop_duplicates(subset = ['state_code','date']) # duplicates need to be added instead of removing
merged['daily_case'] = merged['cases'].diff().fillna(merged['cases'].iloc[0])
merged.head()


print(len(merged))
merged = merged[ ~merged['date'].isin(['2020-04-16'])]
print(len(merged))

merged = merged.drop(['mean_hh_cli_ct_weighted'], axis =1)
len(merged.columns)

features = ['pct_cmnty_cli_weighted', 'pct_self_anosmia_ageusia_weighted', 'pct_hh_cli_weighted', 'pct_hh_fever_weighted', 'pct_self_fever_weighted', 'pct_hh_sore_throat_weighted', 'pct_avoid_contact_all_or_most_time_weighted', 'pct_hh_difficulty_breathing_weighted', 'pct_self_persistent_pain_pressure_in_chest_weighted', 'pct_self_runny_nose_weighted', 'pct_worked_outside_home_weighted', 'pct_self_nausea_vomiting_weighted', 'pct_hh_shortness_of_breath_weighted', 'pct_self_sore_throat_weighted', 'pct_self_difficulty_breathing_weighted', 'pct_asthma_weighted', 'pct_self_shortness_of_breath_weighted', 'pct_hh_cough_weighted', 'pct_self_none_of_above_weighted', 'pct_self_diarrhea_weighted', 'pct_chronic_lung_disease_weighted', 'pct_cancer_weighted', 'pct_self_other_weighted', 'pct_self_tiredness_or_exhaustion_weighted', 'pct_self_cough_weighted', 'pct_no_above_medical_conditions_weighted', 'pct_heart_disease_weighted', 'pct_multiple_medical_conditions_weighted', 'pct_autoimmune_disorder_weighted', 'pct_self_nasal_congestion_weighted', 'pct_kidney_disease_weighted',  'pct_self_multiple_symptoms_weighted', 'pct_self_muscle_joint_aches_weighted', 'pct_high_blood_pressure_weighted', 'pct_diabetes_weighted' ]
len(features)
y = merged['daily_case']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13764 entries, 0 to 13763
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    13764 non-null  object
 1   state   13764 non-null  object
 2   cases   13764 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 322.7+ KB
None
         date state_code  cases
0  2020-03-13         al      6
1  2020-03-14         al     12
2  2020-03-15         al     23
3  2020-03-16         al     29
4  2020-03-17         al     39
7363
num states:  50
7228
7178


# Feature Selection

In [5]:
features_ranked = []
features_copy = features.copy()
i = 1
for fr in (features):
    X_reg_new = SelectKBest(score_func=f_regression, k=1).fit_transform(merged[features_copy],y)
    to_comp = np.squeeze(X_reg_new)
    cols = features
    for c in cols:
        cur_col=np.array(merged[c])
        if np.allclose(to_comp,cur_col):
            selected_feature=c
            features_ranked.append(selected_feature)
            print(i,selected_feature)
            features_copy.remove(selected_feature)
            break
    i +=1

1 pct_cmnty_cli_weighted
2 pct_self_anosmia_ageusia_weighted
3 pct_hh_fever_weighted
4 pct_hh_cli_weighted
5 pct_hh_sore_throat_weighted
6 pct_self_fever_weighted
7 pct_asthma_weighted
8 pct_autoimmune_disorder_weighted
9 pct_self_runny_nose_weighted
10 pct_self_sore_throat_weighted
11 pct_avoid_contact_all_or_most_time_weighted
12 pct_self_none_of_above_weighted
13 pct_self_muscle_joint_aches_weighted
14 pct_self_persistent_pain_pressure_in_chest_weighted
15 pct_chronic_lung_disease_weighted
16 pct_no_above_medical_conditions_weighted
17 pct_self_nasal_congestion_weighted
18 pct_self_multiple_symptoms_weighted
19 pct_cancer_weighted
20 pct_self_nausea_vomiting_weighted
21 pct_multiple_medical_conditions_weighted
22 pct_heart_disease_weighted
23 pct_self_tiredness_or_exhaustion_weighted
24 pct_diabetes_weighted
25 pct_hh_difficulty_breathing_weighted
26 pct_self_other_weighted
27 pct_worked_outside_home_weighted
28 pct_hh_shortness_of_breath_weighted
29 pct_hh_cough_weighted
30 pct_sel

# Model Training

In [7]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time

class Classifier_RESNET:

    def __init__(self, input_shape,  verbose=True, build=True):
        if build == True:
            self.model = self.build_model(input_shape, )
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
      
            self.model.save_weights('model_init.hdf5')
        return

    def build_model(self, input_shape):
        n_feature_maps = 64 * 2

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_y)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        # r= 4 worked better
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=1, padding='same')(conv_y_1)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)


        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=3, padding='same')(conv_y_2)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=5, padding='same')(conv_y_3)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.add([conv_y_1, conv_y_2, conv_y_3])


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL
        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(256, activation='relu')(gap_layer)
        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(128, activation='linear')(gap_layer)

        output_layer = keras.layers.Dense(1, activation='linear')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='mae', optimizer=keras.optimizers.Adam(),
                      metrics=['mae'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=0.0001)

        file_path =  'best_model.hdf5'

        # model_checkpoint = keras.callbacks.ModelCheckpoint('model_{epoch:03d}_{mae:03f}_{val_mae:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
        model_checkpoint = keras.callbacks.ModelCheckpoint(file_path, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
        #                                                   save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit_predict(self, x_train, y_train, x_val, y_val):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = 128
        nb_epochs = 500

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save( 'best_model.hdf5')

        y_pred = self.predict(x_val)

        keras.backend.clear_session()

        return y_pred

    def predict(self, x_test):
        start_time = time.time()
        model_path =  'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        return y_pred

merged = merged.sort_values(by='date', ascending=True)
X_train = merged[:len(merged)*80//100]
X_test = merged[len(merged)*80//100:]
X_train = X_train.sample(frac=1)
y_train = X_train['daily_case']
X_test = X_test.sample(frac=1)
y_test = X_test['daily_case']
print(len(X_train), len(y_test))

X_train = X_train[features_ranked]
X_test = X_test[features_ranked]
print(X_train.shape, X_test.shape)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)


model= Classifier_RESNET(input_shape=(1, 35))
y_pred = model.fit_predict(X_train, y_train, X_test, y_test)

yp_l = y_pred.tolist()
yt_l = y_test.tolist()
abs_sum = 0
pct_sum = 0
for idx in range(len(y_pred)):
    diff = abs(yp_l[idx][0] - yt_l[idx])
    abs_sum += diff
    pct = diff/ (yt_l[idx] + 1)
    pct_sm += pct

rel_error = pct_sm / len(y_pred)
abs_error = abs_sum / len(y_pred)
gb_rel_errors.append(rel_error * 100)
gb_abs_errors.append(abs_error)

print('Test: ', abs_error,rel_error * 100)

y_pred = model.predict(X_train)
yp_l = y_pred.tolist()
yt_l = y_train.tolist()

abs_sum = 0
pct_sum = 0
for idx in range(len(y_pred)):
    diff = abs(yp_l[idx][0] - yt_l[idx])
    abs_sum += diff
    pct = diff/ (yt_l[idx] + 1)
    pct_sm += pct

rel_error = pct_sm / len(y_pred)
abs_error = abs_sum / len(y_pred)
gb_rel_errors.append(rel_error * 100)
gb_abs_errors.append(abs_error)

print('Train: ', abs_error,rel_error * 100)


5742 1436
(5742, 35) (1436, 35)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 35)]      0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 1, 128)       35968       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 1, 128)       512         conv1d_17[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 1, 128)       0           batch_normalization_18[0][0]     
____________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


45/45 [==============================] - 8s 177ms/step - loss: 368.3604 - mae: 368.3604 - val_loss: 641.7217 - val_mae: 641.7217

Epoch 00002: val_loss improved from 1573.27576 to 641.72174, saving model to best_model.hdf5
Epoch 3/500
45/45 [==============================] - 8s 175ms/step - loss: 339.3655 - mae: 339.3655 - val_loss: 737.8402 - val_mae: 737.8402

Epoch 00003: val_loss did not improve from 641.72174
Epoch 4/500
45/45 [==============================] - 8s 176ms/step - loss: 307.7103 - mae: 307.7103 - val_loss: 787.9048 - val_mae: 787.9048

Epoch 00004: val_loss did not improve from 641.72174
Epoch 5/500
45/45 [==============================] - 8s 177ms/step - loss: 297.1655 - mae: 297.1655 - val_loss: 750.3792 - val_mae: 750.3792

Epoch 00005: val_loss did not improve from 641.72174
Epoch 6/500
45/45 [==============================] - 8s 178ms/step - loss: 287.3686 - mae: 287.3686 - val_loss: 792.0226 - val_mae: 792.0226

Epoch 00006: val_loss did not improve from 641.721

KeyboardInterrupt: ignored

### Evaluation

In [8]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time

import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt

class Classifier_RESNET:

    def __init__(self, input_shape,  verbose=True, build=True):
        if build == True:
            self.model = self.build_model(input_shape, )
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
      
            self.model.save_weights('model_init.hdf5')
        return

    def build_model(self, input_shape):
        n_feature_maps = 64 * 2

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_y)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        # r= 4 worked better
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=1, padding='same')(conv_y_1)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)


        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=3, padding='same')(conv_y_2)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=5, padding='same')(conv_y_3)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.add([conv_y_1, conv_y_2, conv_y_3])


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL
        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(256, activation='relu')(gap_layer)
        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(128, activation='linear')(gap_layer)

        output_layer = keras.layers.Dense(1, activation='linear')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='mae', optimizer=keras.optimizers.Adam(),
                      metrics=['mae'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=0.0001)

        file_path =  'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint('model_{epoch:03d}_{mae:03f}_{val_mae:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
        #                                                   save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit_predict(self, x_train, y_train, x_val, y_val):
        if not tf.test.is_gpu_available:
            print('error')
            exit()

        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = 128
        nb_epochs = 500

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save( 'best_model.hdf5')

        y_pred = self.predict(x_val)

        keras.backend.clear_session()

        return y_pred

    def predict(self, x_test):
        start_time = time.time()
        model_path =  'best_model.hdf5'
        # model_path = '/content/model_045_204.676987_362.147278.h5'
        # model_path = '/content/model_118_138.541870_367.545013.h5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        return y_pred


model= Classifier_RESNET(input_shape=(1, 35))



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1, 35)]      0                                            
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 1, 128)       35968       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 1, 128)       512         conv1d_34[0][0]                  
__________________________________________________________________________________________________
activation_30 (Activation)      (None, 1, 128)       0           batch_normalization_36[0][0]     
____________________________________________________________________________________________

In [9]:
# Test scores results are better than the paper
def get_mae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0]-y_true[j])
        abs_sum += diff
    abs_error = abs_sum/len(y_pred)
    print('mae: ', abs_error)
    return abs_error

def get_nmae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0]-y_true[j])
        abs_sum += diff
    n_abs_error = abs_sum/sum(y_true)
    print('nmae: ', n_abs_error)
    return n_abs_error

def get_mre(y_pred, y_true):
    rel_sum = 0
    for j in range(len(y_pred)):
        diff= y_pred[j][0]-y_true[j]
        rel = diff/(y_true[j] +1)
        rel_sum += abs(rel)
    rel_error = rel_sum/len(y_pred)
    print('mre: ', rel_error)
    return rel_error*100
    
print('Test: ')
y_pred = model.predict(X_test)
yp_l = y_pred.tolist()
yt_l = y_test.tolist()

get_mae(yp_l, yt_l)
get_nmae(yp_l, yt_l)
get_mre(yp_l, yt_l)

y_pred = model.predict(X_train)
yp_l = y_pred.tolist()
yt_l = y_train.tolist()

print('Train: ')
get_mae(yp_l, yt_l)
get_nmae(yp_l, yt_l)
get_mre(yp_l, yt_l)


Test: 
mae:  353.89634976181145
nmae:  0.4236068089966552
mre:  2.2123837190399915
Train: 
mae:  117.47233944502246
nmae:  0.15587882390196792
mre:  1.5160377809062364


151.60377809062365

### Global model evaluation on individual states

In [10]:
gdbt_statewise_abs_error_list = []
gdbt_statewise_n_abs_error_list = []
feature_importance_list_gdbt = []

import warnings
warnings.filterwarnings("ignore")

statewise_dl_errors = []
states = list(merged['state_code'].unique())
states.sort()

merged = merged.sort_values(by='date', ascending=True)
X_train = merged[:len(merged)*80//100]
X_test = merged[len(merged)*80//100:]
X_train = X_train.sample(frac=1)
y_train = X_train['daily_case']
X_test = X_test.sample(frac=1)
y_test = X_test['daily_case']
print(len(X_train), len(y_test))

# X_train = X_train[features_ranked]
# X_test = X_test[features_ranked]
print(X_train.shape, X_test.shape)


for state in states:
    X_train_state = X_train[X_train['state_code'] ==state][features_ranked]
    y_train_state =  X_train[X_train['state_code'] ==state]['daily_case']
    X_test_state = X_test[X_test['state_code'] ==state][features_ranked]
    y_test_state =  X_test[X_test['state_code'] ==state]['daily_case']   

    X_train_state = np.expand_dims(X_train_state, axis=1)
    X_test_state = np.expand_dims(X_test_state, axis=1)

    # reg = GradientBoostingRegressor() 
    reg = Classifier_RESNET(input_shape=(1, 35), verbose=False)
    # reg.fit_predict(X_train_state,y_train_state, X_test_state, y_test_state)    
    y_pred= reg.predict(X_test_state)
    yp_l=list(y_pred)
    yt_l=list(y_test_state)

    mae  = get_mae(yp_l,yt_l)
    nmae = get_nmae(yp_l,yt_l) 
    mre  = get_mre(yp_l,yt_l)    

    print(state,mae,nmae,mre, end ='\t') 
    statewise_dl_errors.append([[state, mae, nmae, mre]])

    print ()    
    gdbt_statewise_abs_error_list.append((state,mae,len(y_test_state)))
    gdbt_statewise_n_abs_error_list.append((state,nmae,sum(y_test_state)))    
    # importances = reg.feature_importances_
    # for i in range(len(features_ranked)):
    #    feature_importance_list_gdbt.append([state,features_ranked[i],importances[i]])sum_error = 0

sum_support = 0
sum_error = 0
for tpl in gdbt_statewise_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]

state_model_global_error = sum_error/sum_support
sum_error = 0
sum_support = 0
for tpl in gdbt_statewise_n_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]
state_model_global_error_norm = sum_error/sum_support

print('global level', state_model_global_error,state_model_global_error_norm)

5742 1436
(5742, 41) (1436, 41)
mae:  34.93873430553236
nmae:  0.4731546342160476
mre:  0.7096196168140733
ak 34.93873430553236 0.4731546342160476 70.96196168140733	
mae:  555.7893218994141
nmae:  0.5064503916401548
mre:  0.5197546683620097
al 555.7893218994141 0.5064503916401548 51.975466836200965	
mae:  426.6389475370708
nmae:  0.7947196081572888
mre:  0.7671419284304777
ar 426.6389475370708 0.7947196081572888 76.71419284304777	
mae:  433.0500231291118
nmae:  0.6046406848510527
mre:  0.9428599901392624
az 433.0500231291118 0.6046406848510527 94.28599901392623	
mae:  1742.101870888158
nmae:  0.24492164376688003
mre:  0.20804268261237788
ca 1742.101870888158 0.24492164376688003 20.804268261237787	
mae:  124.18434102911698
nmae:  0.371049296989027
mre:  0.378524632022131
co 124.18434102911698 0.371049296989027 37.8524632022131	
mae:  27.97583096822103
nmae:  0.5216127588854138
mre:  0.5192127335054593
dc 27.97583096822103 0.5216127588854138 51.921273350545924	
mae:  63.467776711781816
n

In [11]:
sum_support = 0
sum_error = 0
for tpl in gdbt_statewise_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]

state_model_global_error = sum_error/sum_support
sum_error = 0
sum_support = 0
for tpl in gdbt_statewise_n_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]
state_model_global_error_norm = sum_error/sum_support

print('global level', state_model_global_error,state_model_global_error_norm)
lis = statewise_dl_errors

sorted(lis, key=lambda x: x[0])

global level 354.11141314911646 0.42362447195822206


[[['ak', 34.93873430553236, 0.4731546342160476, 70.96196168140733]],
 [['al', 555.7893218994141, 0.5064503916401548, 51.975466836200965]],
 [['ar', 426.6389475370708, 0.7947196081572888, 76.71419284304777]],
 [['az', 433.0500231291118, 0.6046406848510527, 94.28599901392623]],
 [['ca', 1742.101870888158, 0.24492164376688003, 20.804268261237787]],
 [['co', 124.18434102911698, 0.371049296989027, 37.8524632022131]],
 [['dc', 27.97583096822103, 0.5216127588854138, 51.921273350545924]],
 [['de', 63.467776711781816, 0.6966825105574295, 113.03187162117551]],
 [['fl', 1304.4033880869547, 0.33364682607138657, 35.022123870079994]],
 [['ga', 721.2096433003743, 0.31568772048690824, 39.10276855212912]],
 [['hi', 278.16956186294556, 1.2539574539276284, 104.10807992566411]],
 [['ia', 338.92837041219076, 0.460104579952293, 42.118432964317584]],
 [['id', 179.01713180541992, 0.5557237121443085, 66.17865262302561]],
 [['il', 836.6678526560465, 0.4230438141252848, 37.84493191986497]],
 [['in', 350.32968292

## Statewise training

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time

def get_mae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0]-y_true[j])
        abs_sum += diff
    abs_error = abs_sum/len(y_pred)
    print('mae: ', abs_error)
    return abs_error

def get_nmae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0]-y_true[j])
        abs_sum += diff
    n_abs_error = abs_sum/sum(y_true)
    print('nmae: ', n_abs_error)
    return n_abs_error

def get_mre(y_pred, y_true):
    rel_sum = 0
    for j in range(len(y_pred)):
        diff= y_pred[j][0]-y_true[j]
        rel = diff/(y_true[j] +1)
        rel_sum += abs(rel)
    rel_error = rel_sum/len(y_pred)
    print('mre: ', rel_error)
    return rel_error*100

class Classifier_RESNET:

    def __init__(self, input_shape,  verbose=True, build=True):
        if build == True:
            self.model = self.build_model(input_shape, )
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
      
            self.model.save_weights('model_init.hdf5')
        return

    def build_model(self, input_shape):
        n_feature_maps = 64 * 2

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_y)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        # conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        # conv_y = keras.layers.BatchNormalization()(conv_y)
        # conv_y = keras.layers.Activation('relu')(conv_y)

        # r= 4 worked better
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=1, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=1, padding='same')(conv_y_1)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_1 = keras.layers.Activation('relu')(conv_y)


        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=3, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=3, padding='same')(conv_y_2)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_2 = keras.layers.Activation('relu')(conv_y)

        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//16, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.Conv1D(filters=(n_feature_maps * 2)//4, kernel_size=5, padding='same')(conv_y_3)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y_3 = keras.layers.Activation('relu')(conv_y)
        conv_y = keras.layers.add([conv_y_1, conv_y_2, conv_y_3])


        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL
        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(256, activation='relu')(gap_layer)
        gap_layer = keras.layers.Dropout(0.5)(gap_layer)
        gap_layer = keras.layers.Dense(128, activation='linear')(gap_layer)

        output_layer = keras.layers.Dense(1, activation='linear')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='mae', optimizer=keras.optimizers.Adam(),
                      metrics=['mae'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=0.0001)

        file_path =  'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(file_path, verbose=0, monitor='val_loss',save_best_only=True, mode='auto')
        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
        #                                                   save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit_predict(self, x_train, y_train, x_val, y_val):
        if not tf.test.is_gpu_available:
            print('error')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        batch_size = 128
        nb_epochs = 50

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save( 'final_model.hdf5')

        y_pred = self.predict(x_val)

        keras.backend.clear_session()

        return y_pred

    def predict(self, x_test):
        start_time = time.time()
        model_path =  'best_model.hdf5'
        # model_path = '/content/model_045_204.676987_362.147278.h5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        return y_pred


merged = merged.sort_values(by='date', ascending=True)
X_train = merged[:len(merged)*80//100]
X_test = merged[len(merged)*80//100:]
X_train = X_train.sample(frac=1)
y_train = X_train['daily_case']
X_test = X_test.sample(frac=1)
y_test = X_test['daily_case']
print(len(X_train), len(y_test))

# X_train = X_train[features_ranked]
# X_test = X_test[features_ranked]
print(X_train.shape, X_test.shape)




model= Classifier_RESNET(input_shape=(1, 35))



In [ ]:
gdbt_statewise_abs_error_list = []
gdbt_statewise_n_abs_error_list = []
feature_importance_list_gdbt = []

import warnings
warnings.filterwarnings("ignore")



statewise_dl_errors = []
states = list(merged['state_code'].unique())

for state in states:
    X_train_state = X_train[X_train['state_code'] ==state][features_ranked]
    y_train_state =  X_train[X_train['state_code'] ==state]['daily_case']
    X_test_state = X_test[X_test['state_code'] ==state][features_ranked]
    y_test_state =  X_test[X_test['state_code'] ==state]['daily_case']   

    X_train_state = np.expand_dims(X_train_state, axis=1)
    X_test_state = np.expand_dims(X_test_state, axis=1)

    # reg = GradientBoostingRegressor() 
    reg = Classifier_RESNET(input_shape=(1, 35), verbose=False)
    reg.fit_predict(X_train_state,y_train_state, X_test_state, y_test_state)    
    y_pred= reg.predict(X_test_state)
    yp_l=list(y_pred)
    yt_l=list(y_test_state)

    mae  = get_mae(yp_l,yt_l)
    nmae = get_nmae(yp_l,yt_l) 
    mre  = get_mre(yp_l,yt_l)    

    print(state,mae,nmae,mre, end ='\t') 
    statewise_dl_errors.append([[state, mae, nmae, mre]])

    print ()    
    gdbt_statewise_abs_error_list.append((state,mae,len(y_test_state)))
    gdbt_statewise_n_abs_error_list.append((state,nmae,sum(y_test_state)))    
    # importances = reg.feature_importances_
    # for i in range(len(features_ranked)):
    #    feature_importance_list_gdbt.append([state,features_ranked[i],importances[i]])sum_error = 0

sum_support = 0
sum_error = 0
for tpl in gdbt_statewise_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]

state_model_global_error = sum_error/sum_support
sum_error = 0
sum_support = 0
for tpl in gdbt_statewise_n_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]
state_model_global_error_norm = sum_error/sum_support

print('global level', state_model_global_error,state_model_global_error_norm)

# CNN Model 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, BatchNormalization
from keras.callbacks import ModelCheckpoint

NB_EPOCHS = 1000  # num of epochs to test for
BATCH_SIZE = 128

## Create our model
model = Sequential()

# 1st layer: input_dim=8, 12 nodes, RELU
model.add(Conv1D(32, 1, activation='relu',input_shape=(1, 35)))
model.add(BatchNormalization())
model.add(Conv1D(4, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(4, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(16, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(4, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(2, 1, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(24, activation='relu'))
# 2nd layer: 8 nodes, RELU
model.add(Dropout(0.5))
model.add(Dense(8,  activation='relu'))
model.add(Dropout(0.5))
# output layer: dim=1, activation sigmoid
model.add(Dense(1,  activation='linear' ))

# Compile the model
model.compile(loss='mean_absolute_error',   
             optimizer='adam',
             metrics=['mae'])

# checkpoint: store the best model
ckpt_model = 'pima-weights.best.hdf5'
checkpoint = ModelCheckpoint(ckpt_model, 
                            monitor='val_mae',
                            verbose=0,
                            save_best_only=True,
                            mode='max')
file_path =  'best_model.hdf5'

model_checkpoint = keras.callbacks.ModelCheckpoint(file_path, 
                                                   verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
callbacks_list = [model_checkpoint]

print('Starting training...')

merged = merged.sort_values(by='date', ascending=True)
X_train = merged[:len(merged)*80//100]
X_test = merged[len(merged)*80//100:]
X_train = X_train.sample(frac=1)
y_train = X_train['daily_case']
X_test = X_test.sample(frac=1)
y_test = X_test['daily_case']
print(len(X_train), len(y_test))

X_train = X_train[features_ranked]
X_test = X_test[features_ranked]
print(X_train.shape, X_test.shape)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)


# train the model, store the results for plotting
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    epochs=1000,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks_list,
                    verbose=1)

# def print_metrics(y_pred, y_gt):

print("[INFO] predicting house prices...")
y_pred = model.predict(X_test)
yp_l = y_pred.tolist()
yt_l = y_test.tolist()
abs_sum = 0
pct_sum = 0
for idx in range(len(y_pred)):
    diff = abs(yp_l[idx][0][0] - yt_l[idx])
    abs_sum += diff
    pct = diff/ (yt_l[idx] + 1)
    pct_sm += pct

rel_error = pct_sm / len(y_pred)
abs_error = abs_sum / len(y_pred)
gb_rel_errors.append(rel_error * 100)
gb_abs_errors.append(abs_error)

print('Test: ', abs_error,rel_error * 100)

y_pred = model.predict(X_train)
yp_l = y_pred.tolist()
yt_l = y_train.tolist()

abs_sum = 0
pct_sum = 0
for idx in range(len(y_pred)):
    diff = abs(yp_l[idx][0][0] - yt_l[idx])
    abs_sum += diff
    pct = diff/ (yt_l[idx] + 1)
    pct_sm += pct

rel_error = pct_sm / len(y_pred)
abs_error = abs_sum / len(y_pred)
gb_rel_errors.append(rel_error * 100)
gb_abs_errors.append(abs_error)

print('Train: ', abs_error,rel_error * 100)


In [ ]:
def get_mae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0][0]-y_true[j])
        abs_sum += diff
    abs_error = abs_sum/len(y_pred)
    print('mae: ', abs_error)
    return abs_error

def get_nmae(y_pred, y_true):
    abs_sum = 0
    for j in range(len(y_pred)):
        diff=abs(y_pred[j][0][0]-y_true[j])
        abs_sum += diff
    n_abs_error = abs_sum/sum(y_true)
    print('nmae: ', n_abs_error)
    return n_abs_error

def get_mre(y_pred, y_true):
    rel_sum = 0
    for j in range(len(y_pred)):
        diff= y_pred[j][0][0]-y_true[j]
        rel = diff/(y_true[j] +1)
        rel_sum += abs(rel)
    rel_error = rel_sum/len(y_pred)
    print('mre: ', rel_error)
    return rel_error*100
    
print('Test: ')
# model_path = '/content/cnn_model_885_407.555481_459.086090.h5'
# model_path = 'cnn_model_283_420.091034_462.834656.h5'
model_path = file_path
model = keras.models.load_model(model_path)
y_pred = model.predict(X_test)
yp_l = y_pred.tolist()
yt_l = y_test.tolist()

get_mae(yp_l, yt_l)
get_nmae(yp_l, yt_l)
get_mre(yp_l, yt_l)

y_pred = model.predict(X_train)
yp_l = y_pred.tolist()
yt_l = y_train.tolist()

print('Train: ')
get_mae(yp_l, yt_l)
get_nmae(yp_l, yt_l)
get_mre(yp_l, yt_l)


### state wise training (local models)



In [ ]:
import warnings
warnings.filterwarnings("ignore")
from keras import Sequential
from keras.layers import *

gdbt_statewise_abs_error_list = []
gdbt_statewise_n_abs_error_list = []
feature_importance_list_gdbt = []
BATCH_SIZE = 128

statewise_dl_errors = []
states = list(merged['state_code'].unique())
states.sort()

merged = merged.sort_values(by='date', ascending=True)
X_train = merged[:len(merged)*80//100]
X_test = merged[len(merged)*80//100:]
X_train = X_train.sample(frac=1)
y_train = X_train['daily_case']
X_test = X_test.sample(frac=1)
y_test = X_test['daily_case']
print(len(X_train), len(y_test))

# X_train = X_train[features_ranked]
# X_test = X_test[features_ranked]
print(X_train.shape, X_test.shape)

def train_cnn(X_train_state, y_train_state, X_test_state, y_test_state):
    ## Create our model
    model = Sequential()

    # 1st layer: input_dim=8, 12 nodes, RELU
    model.add(Conv1D(32, 1, activation='relu',input_shape=(1, 35)))
    model.add(BatchNormalization())
    model.add(Conv1D(4, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(32, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(4, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(16, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(4, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(2, 1, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(24, activation='relu'))
    # 2nd layer: 8 nodes, RELU
    model.add(Dropout(0.5))
    model.add(Dense(8,  activation='relu'))
    model.add(Dropout(0.5))
    # output layer: dim=1, activation sigmoid
    model.add(Dense(1,  activation='linear' ))

    # Compile the model
    model.compile(loss='mean_absolute_error',   
                optimizer='adam',
                metrics=['mae'])

    # checkpoint: store the best model
    """
    ckpt_model = 'pima-weights.best.hdf5'
    checkpoint = ModelCheckpoint(ckpt_model, 
                                monitor='val_mae',
                                verbose=0,
                                save_best_only=True,
                                mode='max')
    file_path =  'best_model.hdf5'
    """
    model_checkpoint = keras.callbacks.ModelCheckpoint('best_model.hdf5', 
                                                    verbose=0, monitor='val_loss',save_best_only=True, mode='auto')
    callbacks_list = [model_checkpoint]

    # print('Starting training...')

    # train the model, store the results for plotting
    history = model.fit(X_train_state,
                        y_train_state,
                        validation_data=(X_test_state, y_test_state),
                        epochs=100,
                        batch_size=BATCH_SIZE,
                        callbacks=callbacks_list,
                        verbose=0)
    return model

for state in states:
    X_train_state = X_train[X_train['state_code'] ==state][features_ranked]
    y_train_state =  X_train[X_train['state_code'] ==state]['daily_case']
    X_test_state = X_test[X_test['state_code'] ==state][features_ranked]
    y_test_state =  X_test[X_test['state_code'] ==state]['daily_case']   

    X_train_state = np.expand_dims(X_train_state, axis=1)
    X_test_state = np.expand_dims(X_test_state, axis=1)

    # reg = GradientBoostingRegressor() 
    # reg = Classifier_RESNET(input_shape=(1, 35), verbose=False)
    train_cnn(X_train_state, y_train_state, X_test_state, y_test_state)

    
    model_path = 'best_model.hdf5'
    reg = keras.models.load_model(model_path)
    # reg.fit_predict(X_train_state,y_train_state, X_test_state, y_test_state)    
    y_pred= reg.predict(X_test_state)
    yp_l=list(y_pred)
    yt_l=list(y_test_state)

    mae  = get_mae(yp_l,yt_l)
    nmae = get_nmae(yp_l,yt_l) 
    mre  = get_mre(yp_l,yt_l)    

    print(state,mae,nmae,mre, end ='\t') 
    statewise_dl_errors.append([[state, mae, nmae, mre]])

    print ()    
    gdbt_statewise_abs_error_list.append((state,mae,len(y_test_state)))
    gdbt_statewise_n_abs_error_list.append((state,nmae,sum(y_test_state)))    
    # importances = reg.feature_importances_
    # for i in range(len(features_ranked)):
    #    feature_importance_list_gdbt.append([state,features_ranked[i],importances[i]])sum_error = 0

sum_support = 0
sum_error = 0
for tpl in gdbt_statewise_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]

state_model_global_error = sum_error/sum_support
sum_error = 0
sum_support = 0
for tpl in gdbt_statewise_n_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]
state_model_global_error_norm = sum_error/sum_support

print('global level', state_model_global_error,state_model_global_error_norm)

sum_support = 0
sum_error = 0
for tpl in gdbt_statewise_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]

state_model_global_error = sum_error/sum_support
sum_error = 0
sum_support = 0
for tpl in gdbt_statewise_n_abs_error_list: 
    sum_error += tpl[1] * tpl[2]
    sum_support += tpl[2]
state_model_global_error_norm = sum_error/sum_support

print('global level', state_model_global_error,state_model_global_error_norm)
lis = statewise_dl_errors

sorted(lis, key=lambda x: x[0])
